In [1]:
import ensembl_rest
import numpy as np
import pandas as pd

In [2]:
def get_length_aa(enst_list: list):
    length_aa = []
    enst_info = {}
    
    enst_chunks = (enst_list[pos:pos + 100] for pos in range(0, len(enst_list), 100))
    
    for i in enst_chunks:
        enst_info.update(ensembl_rest.lookup_post(species='human', params={'expand': True, 'ids': i}))
    
    for e in enst_list:
        try:
            length_aa.append(enst_info[e]['Translation']['length'])
        except TypeError:
            length_aa.append(np.nan)
        
    return length_aa

In [9]:
ensembl_annotation = pd.read_csv('../data/processed/ensembl_annotation_trs_uniprot_20220429.csv', low_memory=False)

In [4]:
with open('../data/raw/nextprot_050222_missing_pe2_090422.txt') as f:
    missing_ac_list = f.read().splitlines()

In [5]:
missing_df = pd.concat([ensembl_annotation[ensembl_annotation['uniprot_base']==ac] for ac in missing_ac_list])

In [6]:
missing_df = missing_df[['uniprot_isoform', 'ensembl_gene_name', 'ensembl_trs_id', 'ensembl_is_canonical', 'trs_length_bp']]

In [7]:
missing_df.loc[:, 'protein_length_aa'] = missing_df['ensembl_trs_id'].map(get_length_aa(missing_df['ensembl_trs_id'].to_list()))

In [8]:
missing_df = missing_df.reset_index(drop=True)

In [10]:
hek = pd.read_csv('../data/processed/hek293t_va_20220525.csv', index_col=0)['HEK293T']

In [12]:
missing_df['hek293t_tpm'] = missing_df['ensembl_trs_id'].map(hek)

In [75]:
missing_df = missing_df.fillna(0)

In [76]:
missing_df

,uniprot_isoform,ensembl_gene_name,ensembl_trs_id,ensembl_is_canonical,trs_length_bp,protein_length_aa,hek293t_tpm,caco2_tpm,hacat_tpm,hela_tpm,hepg2_tpm
0,A0A075B6N3-1,TRBV24-1,ENST00000390397,True,381,115,0.000000,0.000000,0.000000,0.000000,0.000000
1,A0A075B6N4-1,TRBV25-1,ENST00000390398,True,381,114,0.000000,0.000000,0.000000,0.000000,0.000000
2,A0A075B6T7-1,TRAV6,ENST00000390428,True,404,132,0.000000,0.000000,0.000000,0.000000,0.000000
3,A0A075B6U4-1,TRAV7,ENST00000390429,True,337,112,0.000000,0.000000,0.000000,0.000000,0.000000
4,A0A075B6V5-1,TRAV36DV7,ENST00000390463,True,356,113,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
1508,Q9Y5P0-1,OR51B4,ENST00000380224,True,933,310,0.023247,0.017786,1.773046,0.000000,0.016616
1509,Q9Y6U7-1,RNF215,ENST00000382363,True,2011,377,5.529197,1.647353,0.719113,3.667213,1.321955
1510,S4R3P1-1,MTRNR2L13,ENST00000604093,True,1445,24,0.000000,0.004596,0.000000,0.000000,0.000000
1511,S4R3Y5-1,MTRNR2L11,ENST00000604646,True,1552,24,0.000000,0.008883,0.000000,0.000000,0.000000


In [83]:
missing_good2 = missing_good[(missing_good > 4).any(1)].index.to_list()

In [19]:
missing_df_select = missing_df[(missing_df['protein_length_aa'] >= 90) & (missing_df['uniprot_isoform'].isin(missing_good))]

In [63]:
missing_df_select = missing_df_select.set_index(['ensembl_gene_name', 'uniprot_isoform', 'protein_length_aa', 'ensembl_trs_id'], drop=True)

In [59]:
missing_df_select = missing_df_select.reset_index()

In [61]:
missing_df_select = missing_df_select.sort_values('ensembl_gene_name')

In [64]:
missing_df_select

ensembl_is_canonical  \
ensembl_gene_name uniprot_isoform protein_length_aa ensembl_trs_id                          
AGAP9             Q5VTM2-2        658               ENST00000452145                  True   
ASAH2B            P0C7U1-1        165               ENST00000643851                 False   
                                                    ENST00000374006                 False   
                  P0C7U1-2        160               ENST00000374007                 False   
                                                    ENST00000647317                  True   
C1orf54           Q8WWF1-1        131               ENST00000369099                  True   
                                                    ENST00000369102                 False   
CNPY1             Q3B7I2-1        92                ENST00000321736                 False   
                                                    ENST00000406197                 False   
CYB5RL            Q6IPT4-4        247               ENST00000287899                 False   
DDTL              A6NHG4-1        134               ENST00000621454                  True   
                                                    ENST00000215770                  True   
EOLA2             Q96DE9-1        158               ENST00000355203                 False   
                                                    ENST00000370404                 False   
                                                    ENST00000370406                  True   
FAM72C            H0Y354-1        149               ENST00000584486                  True   
GAGE13            Q4V321-1        117               ENST00000612958                  True   
GOLGA8A           A7E2F4-3        603               ENST00000359187                  True   
                  A7E2F4-1        631               ENST00000432566                 False   
LRRC75B           Q2VPJ9-1        315               ENST00000318753                  True   
MOSMO             Q8NHV5-1        167               ENST00000542527                  True   
NBPF11            Q86T75-1        865               ENST00000614015                 False   
                                                    ENST00000615281                 False   
NBPF12            Q5TAG4-1        1457              ENST00000617931                  True   
                                                    ENST00000617844                 False   
NBPF15            Q8N660-1        670               ENST00000581897                  True   
                                                    ENST00000577412                 False   
                                                    ENST00000488031                 False   
NPIPA1            Q9UND3-1        350               ENST00000328085                  True   
PPM1N             Q8N819-3        112               ENST00000401705                 False   
                                                    ENST00000396737                 False   
                                                    ENST00000396735                 False   
RNF215            Q9Y6U7-1        377               ENST00000382363                  True   
TLCD5             Q6ZRR5-1        245               ENST00000375095                  True   
TMEM81            Q6P7N7-1        255               ENST00000367167                  True   
TSPAN17           Q96FV3-2        188               ENST00000298564                 False   
                  Q96FV3-3        263               ENST00000515708                 False   
                  Q96FV3-4        329               ENST00000508164                  True   

                                                                     trs_length_bp  \
ensembl_gene_name uniprot_isoform protein_length_aa ensembl_trs_id                   
AGAP9             Q5VTM2-2        658               ENST00000452145           2387   
ASAH2B            P0C7U1-1        165               ENST00000643851           5005   
    

In [55]:
missing_genes = ensembl_annotation[ensembl_annotation['ensembl_gene_name'].isin(missing_df_select.index.get_level_values(0))]

In [65]:
missing_df_select.to_excel('../reports/missing_hek_select_20220518.xlsx')

In [106]:
missing_df.query('ensembl_gene_name == "TLCD5"')

,uniprot_isoform,ensembl_gene_name,ensembl_trs_id,ensembl_is_canonical,trs_length_bp,protein_length_aa
208329,Q6ZRR5-1,TLCD5,ENST00000375095,True,3980,245
208330,Q6ZRR5-3,TLCD5,ENST00000314475,False,1434,267
208331,Q6ZRR5-4,TLCD5,ENST00000529187,False,3930,148


In [56]:
missing_genes['TPM'] = missing_genes['ensembl_trs_id'].map(hek)

C:\Users\Dynev\AppData\Local\Temp\ipykernel_16876\344735713.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_genes['TPM'] = missing_genes['ensembl_trs_id'].map(hek)


In [38]:
def get_enst_name(enst_list: list):
    length_aa = []
    enst_info = {}
    
    enst_chunks = (enst_list[pos:pos + 100] for pos in range(0, len(enst_list), 100))
    
    for i in enst_chunks:
        enst_info.update(ensembl_rest.lookup_post(species='human', params={'expand': True, 'ids': i}))
    
    for e in enst_list:
        try:
            length_aa.append(enst_info[e]['display_name'])
        except TypeError:
            length_aa.append(np.nan)
        
    return length_aa

In [33]:
get_enst_name(['ENST00000452145'])

ENST00000452145    AGAP9-201
dtype: object

In [51]:
missing_genes = missing_genes[(missing_genes['TPM'] > 1)]

In [57]:
missing_genes['ensembl_trs_name'] = get_enst_name(missing_genes['ensembl_trs_id'].to_list())

C:\Users\Dynev\AppData\Local\Temp\ipykernel_16876\426781810.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_genes['ensembl_trs_name'] = get_enst_name(missing_genes['ensembl_trs_id'].to_list())


In [58]:
missing_genes.to_excel('../reports/missing_genes_other.xlsx', index=False)

In [54]:
missing_genes[missing_genes['ensembl_gene_name'] == 'DDTL']

,ensembl_gene_name,ensembl_gene_id,ensembl_trs_id,ensembl_is_canonical,trs_type,trs_length_bp,ensembl_protein_id,uniprot_base,uniprot_isoform,TPM,ensembl_trs_name
45937,DDTL,ENSG00000275758,ENST00000621454,True,protein_coding,1545,ENSP00000479063,A6NHG4,A6NHG4-1,2.481337,DDTL-202
45938,DDTL,ENSG00000099974,ENST00000215770,True,protein_coding,1582,ENSP00000215770,A6NHG4,A6NHG4-1,3.418826,DDTL-201


In [47]:
missing_genes

,ensembl_gene_name,ensembl_gene_id,ensembl_trs_id,ensembl_is_canonical,trs_type,trs_length_bp,ensembl_protein_id,uniprot_base,uniprot_isoform,TPM,ensembl_trs_name
5288,AGAP9,ENSG00000204172,ENST00000452145,True,protein_coding,2387,ENSP00000392206,Q5VTM2,Q5VTM2-2,17.906937,AGAP9-201
13420,ASAH2B,ENSG00000204147,ENST00000643851,False,protein_coding,5005,ENSP00000495463,P0C7U1,P0C7U1-1,1.375991,ASAH2B-205
13421,ASAH2B,ENSG00000204147,ENST00000374006,False,protein_coding,632,ENSP00000363118,P0C7U1,P0C7U1-1,3.907073,ASAH2B-201
13422,ASAH2B,ENSG00000204147,ENST00000374007,False,protein_coding,3715,ENSP00000363119,P0C7U1,P0C7U1-2,3.989175,ASAH2B-202
13423,ASAH2B,ENSG00000204147,ENST00000647317,True,protein_coding,5145,ENSP00000496089,P0C7U1,P0C7U1-2,2.919516,ASAH2B-207
21882,C1orf54,ENSG00000118292,ENST00000369099,True,protein_coding,509,ENSP00000358095,Q8WWF1,Q8WWF1-1,9.431817,C1orf54-202
21883,C1orf54,ENSG00000118292,ENST00000369098,False,protein_coding,418,ENSP00000358094,NaN,NaN,8.450435,C1orf54-201
36991,CNPY1,ENSG00000146910,ENST00000406197,False,protein_coding,560,ENSP00000384514,Q3B7I2,Q3B7I2-1,19.871630,CNPY1-202
36992,CNPY1,ENSG00000146910,ENST00000321736,False,protein_coding,2378,ENSP00000317439,Q3B7I2,Q3B7I2-1,8.023424,CNPY1-201
43140,CYB5RL,ENSG00000215883,ENST00000287899,False,protein_coding,5777,ENSP00000287899,Q6IPT4,Q6IPT4-4,5.427784,CYB5RL-201


In [90]:
missing_df_select2 = missing_df[(missing_df['protein_length_aa'] >= 90) & (missing_df['uniprot_isoform'].isin(missing_good2))]

In [91]:
missing_df_select2

,uniprot_isoform,ensembl_gene_name,ensembl_trs_id,ensembl_is_canonical,trs_length_bp,protein_length_aa,hek293t_tpm,caco2_tpm,hacat_tpm,hela_tpm,hepg2_tpm
25,A0A096LP55-1,UQCRHL,ENST00000483273,True,2180,91,0.430177,4.057305,0.481907,0.107877,0.055164
196,A6NC51-1,TMEM150B,ENST00000326652,True,983,233,0.000000,27.130582,0.000000,0.000000,0.874709
197,A6NCE7-1,MAP1LC3B2,ENST00000556529,True,818,125,1.796652,4.891440,0.803786,3.268436,2.478418
229,A6NHG4-1,DDTL,ENST00000621454,True,1545,134,2.481337,0.000000,0.000000,0.000000,0.000000
230,A6NHG4-1,DDTL,ENST00000215770,True,1582,134,3.418826,4.075057,0.000000,1.718439,0.000000
278,A7E2F4-1,GOLGA8A,ENST00000432566,False,4577,631,5.411162,1.461287,1.214195,19.582925,1.519552
279,A7E2F4-3,GOLGA8A,ENST00000359187,True,5777,603,5.787899,1.638568,3.221302,10.371453,2.729271
282,A8K830-1,COLCA2,ENST00000614153,False,1264,154,0.000000,0.366155,0.000000,0.000000,0.812767
283,A8K830-1,COLCA2,ENST00000398035,False,1414,154,0.000000,1.380386,0.412422,0.000000,2.516043
284,A8K830-1,COLCA2,ENST00000526216,False,1332,154,0.000000,1.152619,0.132022,0.000000,2.554955


In [6]:
hacat = pd.read_csv('../data/processed/va_cellline_expression_20220517.csv')

In [8]:
hacat#[hacat['gene_name'] == 'MECP2']

,transcript_id,A549,CACO2,HACAT,HEK293T,HELA,HEPG2,HUH7,MCF7,SAOS2,SKBR3,U2OS
0,ENST00000440075,0.256721,9.788971,0.407386,0.000000,0.000000,0.159880,0.065040,0.000000,0.000000,0.184198,0.000000
1,ENST00000398101,0.433563,0.367757,0.000000,0.058994,0.021996,0.000000,0.222245,0.066869,1.224826,1.930838,3.093246
2,ENST00000261219,0.175636,0.417713,0.674782,0.068071,0.000000,0.154386,0.116461,0.293679,0.791766,1.853371,0.643787
3,ENST00000537360,0.548286,0.265523,0.044893,0.038378,0.054987,0.070682,0.033041,0.000000,0.108896,0.211980,0.049342
4,ENST00000367067,1.626066,1.568156,0.634107,0.000000,27.164890,0.620189,0.172326,0.980916,2.233141,1.346990,1.843781
...,...,...,...,...,...,...,...,...,...,...,...,...
57157,ENST00000674031,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.531759
57158,ENST00000674194,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.039345
57159,ENST00000675459,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.021859
57160,ENST00000675842,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.007865


In [10]:
trs_mecp2 = ensembl_annotation[ensembl_annotation['ensembl_gene_name'] == 'MECP2']['ensembl_trs_id'].to_list()

In [12]:
h2 = hacat[hacat['transcript_id'].isin(trs_mecp2)]

In [13]:
h2['transcript_id'] = h2['transcript_id'].map(ensembl_annotation.set_index('ensembl_trs_id'))

,transcript_id,A549,CACO2,HACAT,HEK293T,HELA,HEPG2,HUH7,MCF7,SAOS2,SKBR3,U2OS
1776,ENST00000628176,6.072629,2.279038,11.465819,21.053546,4.024701,5.406935,3.005317,0.924718,0.785814,3.808402,8.318872
6710,ENST00000407218,2.307239,2.009729,1.589735,5.789601,1.398218,1.079597,0.000000,0.015633,0.000000,1.241592,1.215075
21196,ENST00000303391,8.261495,1.627757,3.161258,7.486567,4.219277,2.187321,0.683029,6.027829,13.386849,8.725702,9.450176
21197,ENST00000453960,0.349907,1.580946,1.232179,9.151195,1.810952,1.031644,1.771724,2.039655,7.562515,0.996862,1.579213
